import pickle
# ^^^ pyforest auto-imports - don't write above this line
# Imports

In [1]:
from haversine import haversine, Unit
import copy
import re
import folium
from functions import *

## Loading Data

In [2]:
nyc_attractions = pd.read_csv("./data/nyc_attractions", index_col=0)
with open('coords', 'rb') as f:
    coords = pickle.load(f)

<IPython.core.display.Javascript object>

# Making Grid of Distances

In [3]:
# adding hotel
if [40.764712, -73.974485] not in coords:
    coords.insert(0, [40.764712, -73.974485])

In [4]:
dist_matrix = get_distance(coords)

In [5]:
len(dist_matrix)

21

# Insert into OR-tools (TSP)
original py file: https://github.com/google/or-tools/blob/stable/ortools/constraint_solver/samples/tsp_cities.py

**Note**: This code was adapted from the above python file developed by Google OR-Tools. License can be found here: https://www.apache.org/licenses/LICENSE-2.0

In [ ]:
# [START import]
from __future__ import print_function
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
# [END import]

# [START data_model]
def create_data_model(input_distance_matrix):
    """Stores the data for the problem."""
    data = {}
#     data['distance_matrix'] = [
#         [0, 2451, 713, 1018, 1631, 1374, 2408, 213, 2571, 875, 1420, 2145, 1972],
#         [2451, 0, 1745, 1524, 831, 1240, 959, 2596, 403, 1589, 1374, 357, 579],
#         [713, 1745, 0, 355, 920, 803, 1737, 851, 1858, 262, 940, 1453, 1260],
#         [1018, 1524, 355, 0, 700, 862, 1395, 1123, 1584, 466, 1056, 1280, 987],
#         [1631, 831, 920, 700, 0, 663, 1021, 1769, 949, 796, 879, 586, 371],
#         [1374, 1240, 803, 862, 663, 0, 1681, 1551, 1765, 547, 225, 887, 999],
#         [2408, 959, 1737, 1395, 1021, 1681, 0, 2493, 678, 1724, 1891, 1114, 701],
#         [213, 2596, 851, 1123, 1769, 1551, 2493, 0, 2699, 1038, 1605, 2300, 2099],
#         [2571, 403, 1858, 1584, 949, 1765, 678, 2699, 0, 1744, 1645, 653, 600],
#         [875, 1589, 262, 466, 796, 547, 1724, 1038, 1744, 0, 679, 1272, 1162],
#         [1420, 1374, 940, 1056, 879, 225, 1891, 1605, 1645, 679, 0, 1017, 1200],
#         [2145, 357, 1453, 1280, 586, 887, 1114, 2300, 653, 1272, 1017, 0, 504],
#         [1972, 579, 1260, 987, 371, 999, 701, 2099, 600, 1162, 1200, 504, 0],
#     ]  # yapf: disable
    data['distance_matrix'] = input_distance_matrix
    data['num_vehicles'] = 1
    data['depot'] = 0
    return data
    # [END data_model]


# [START solution_printer]
def print_solution(manager, routing, solution):
    """Prints solution on console."""
    total_miles = solution.ObjectiveValue() / 1000
    print('Objective: {} miles'.format(total_miles))
    index = routing.Start(0)
    plan_output = 'Route for vehicle 0:\n'
    route_distance = 0
    while not routing.IsEnd(index):
        plan_output += ' {} ->'.format(manager.IndexToNode(index))
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
    plan_output += ' {}\n'.format(manager.IndexToNode(index))
    print(plan_output)
    plan_output += 'Route distance: {}miles\n'.format(route_distance)
    return total_miles, plan_output
    # [END solution_printer]


def main():
    """Entry point of the program."""
    # Instantiate the data problem.
    # [START data]
    data = create_data_model(dist_matrix)
    # [END data]

    # Create the routing index manager.
    # [START index_manager]
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])
    # [END index_manager]

    # Create Routing Model.
    # [START routing_model]
    routing = pywrapcp.RoutingModel(manager)

    # [END routing_model]

    # [START transit_callback]
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    # [END transit_callback]

    # Define cost of each arc.
    # [START arc_cost]
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
    # [END arc_cost]

    # Setting first solution heuristic.
    # [START parameters]
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    # [END parameters]

    # Solve the problem.
    # [START solve]
    solution = routing.SolveWithParameters(search_parameters)
    # [END solve]

    # Print solution on console.
    # [START print_solution]
    if solution:
        miles, route_list = print_solution(manager, routing, solution)
    route_list_text_match = re.match(".+vehicle\s0:\\n\s(.+0)\\n", route_list)
    stops_as_text = route_list_text_match.group(1)
    list_of_stops = stops_as_text.split(" -> ")
    
    return miles, list_of_stops
    # [END print_solution]


if __name__ == '__main__':
    miles, list_of_stops = main()
    
# [END program]

# Interpret and Plot results

##  Function Outputs

In [ ]:
miles

In [ ]:
list_of_stops[0:2]

## Translate Outputs to place names

In [ ]:
nyc_attractions.reset_index(inplace=True)

In [ ]:
correct_order_of_sites = []
for x in list_of_stops:
    int_x = int(x)
    if int_x >= 1:
#         print(nyc_attractions.iloc[int_x-1]['Site'])
        correct_order_of_sites.append(nyc_attractions.iloc[int_x-1]['Site'])

In [ ]:
nyc_attractions.head(2)

In [ ]:
len(correct_order_of_sites)

## Itinerary

In [ ]:
itinerary_df = pd.DataFrame()
for x in list_of_stops:
    int_x = int(x)
    if int_x >= 1:
        pd.concat([itinerary_df, nyc_attractions.iloc[int_x-1]])

In [ ]:
nyc_attractions.shape

In [ ]:
# using concat would be equally as efficient 
itinerary_df = nyc_attractions.copy()

In [ ]:
itinerary_df.head(2)

In [ ]:
# add new column
itinerary_df['fastest_order'] = [correct_order_of_sites.index(x) + 1 for x in itinerary_df.Site]

### Adding hotels to itin_df

In [ ]:
hotel_dict = {col: None for col in itinerary_df.columns}

In [ ]:
first_row = ['Plaza Hotel (start)', 40.764712, -73.974485, 0]
for idx, key in enumerate(hotel_dict.keys()):
    hotel_dict[key] = first_row[idx]
itinerary_df = itinerary_df.append(hotel_dict, ignore_index=True)

In [ ]:
itinerary_df.shape

In [ ]:
hotel_dict2 = hotel_dict

In [ ]:
second_row = ['Plaza Hotel (end)', 40.764712, -73.974485, 21]
for idx, key in enumerate(hotel_dict2.keys()):
    hotel_dict2[key] = second_row[idx]
itinerary_df = itinerary_df.append(hotel_dict2, ignore_index=True)

### Sorting by fastest order

In [ ]:
itinerary_df.sort_values(by='fastest_order', inplace=True)

In [ ]:
itinerary_df.set_index('fastest_order', inplace=True)

### Making coords into a column

In [ ]:
itinerary_df['coords'] = list(zip(itinerary_df['Latitude'], itinerary_df['Longitude']))

In [ ]:
itinerary_df.head()

# Mapping Results

## Creating Base Map 

In [ ]:
# getting mid lat and long 
mean_lat = itinerary_df['Latitude'].mean()
mean_lng = itinerary_df['Longitude'].mean()

In [ ]:
base_map = folium.Map(location=[mean_lat, mean_lng], tiles="CartoDB positron", zoom_start=12)

## Adding Locations

In [ ]:
for num in range(1, len(itinerary_df)-1):
    folium.Marker(itinerary_df['coords'][num], popup=itinerary_df['Site'][num]).add_to(base_map)
# separate one for hotel
folium.Marker(itinerary_df['coords'][0], popup=itinerary_df['Site'][0], icon=folium.Icon(color='black')).add_to(base_map)

In [ ]:
base_map

## Adding Lines

In [ ]:
folium.PolyLine(itinerary_df['coords'], color="red", weight=7.5, opacity=1).add_to(base_map)

In [ ]:
base_map

In [ ]:
# saving map 
# base_map.save("Top_NYC_attractions_w_fastest_route.html")